## Script is designed to create the taxonomy table file

Output is: <br>

1. <i> File "Tax_gtdb.csv" <br></i>
Taxonomy table file. Taxonomy was taken from last GTDB release (release207/207.0) <br>
https://data.ace.uq.edu.au/public/gtdb/data/releases/release207/207.0/ <br>
<br>
2. <i> File "Tax_gtdb-ncbi.csv" <br></i>
Taxonomy table file. The table includes the gtdb taxonomy and ncbi names for all species for which I find a correspondence between dtdb and ncbi spesies names. 
Gtdb species names vs ncbi species names match files were taken from here: <br>
https://data.gtdb.ecogenomic.org/releases/latest/auxillary_files/


In [2]:
# import packages
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from matplotlib_venn import venn3
import pandas as pd
import seaborn as sns 
import os
import glob
import plotly.express as px
import numpy as np

## Tax_gtdb

In [7]:
path='/Users/katerynapantiukh/Documents/1MyDisk/Code/Microbiome/'
colnames = ['dom','phylum','class','order','family','genus','sp']

#bac = pd.read_csv('input/bac120_taxonomy_r207.tsv', sep=';', names=colnames, header=None)
#arh = pd.read_csv('input/ar53_taxonomy_r207.tsv', sep=';', names=colnames, header=None)
bac = pd.read_csv(str(path)+'/gtdb_vizualization/input/bac120_taxonomy_r214.tsv', sep=';', names=colnames, header=None)
arh = pd.read_csv(str(path)+'gtdb_vizualization/input/ar53_taxonomy_r214.tsv', sep=';', names=colnames, header=None)

# modify bac120 table
bac['domain'] = bac['dom'].str.split('__', expand=True)[1]
bac['phylum'] = bac['phylum'].str.replace(r'p__', '')
bac['class'] = bac['class'].str.replace(r'c__', '')
bac['order'] = bac['order'].str.replace(r'o__', '')
bac['family'] = bac['family'].str.replace(r'f__', '')
bac['genus'] = bac['genus'].str.replace(r'g__', '')
bac['sp'] = bac['sp'].str.replace(r's__', '')
bac = bac[['domain','phylum','class','order','family','genus','sp']]
bac = bac.drop_duplicates()

# modify arch table
arh['domain'] = arh['dom'].str.split('__', expand=True)[1]
arh['phylum'] = arh['phylum'].str.replace(r'p__', '')
arh['class'] = arh['class'].str.replace(r'c__', '')
arh['order'] = arh['order'].str.replace(r'o__', '')
arh['family'] = arh['family'].str.replace(r'f__', '')
arh['genus'] = arh['genus'].str.replace(r'g__', '')
arh['sp'] = arh['sp'].str.replace(r's__', '')
arh = arh[['domain','phylum','class','order','family','genus','sp']]
arh = arh.drop_duplicates()

frames = [bac,arh]
gtdb = pd.concat(frames)

gtdb.to_excel('Tax_gtdb.xlsx', index=False)
gtdb.head(2)

,domain,phylum,class,order,family,genus,sp
0,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia,Escherichia coli
33849,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Klebsiella,Klebsiella pneumoniae


In [8]:
print('Total bacteria species number in GTDB is ',len(gtdb.loc[gtdb['domain'] == 'Bacteria']))

Total bacteria species number in GTDB is  80789


## Tax_gtdb-ncbi

In [4]:
# исходный файл соотыетвия
convB = pd.read_excel('input/gtdb_vs_ncbi_bacteria.xlsx', sheet_name='names')

convB['top_sp_ncbi'] = convB['List of NCBI species'].str.split(',', expand=True)[0]
convB['GTDB R207 species'] = convB['GTDB R207 species'].str.replace(r's__', '')
convB['top_sp_ncbi'] = convB['top_sp_ncbi'].str.replace(r's__', '')
convB.rename(columns = {'GTDB R207 species':'sp_gtdb'}, inplace = True)
new = convB.top_sp_ncbi.str.rsplit(' ', 1, expand=True)\
  .rename(columns=lambda x: 'col{}'.format(x + 1))
convB['sp_ncbi'] = new['col1']

convB = convB[['sp_gtdb','sp_ncbi']]
convB = convB.loc[convB['sp_ncbi'] != "(g__)"]

convB.head(2)

,sp_gtdb,sp_ncbi
0,Zymomonas pomaceae,Zymomonas mobilis
1,Zymomonas mobilis_B,Zymomonas mobilis


In [11]:
rel = '214'
convB = pd.read_excel('input/gtdb_vs_ncbi_r'+str(rel)+'_bacteria.xlsx', sheet_name='Species')
convB

,GTDB R214 species,Number of genomes,Number of NCBI species,List of NCBI species
0,s__Escherichia coli,33849,12,"s__Escherichia coli 91.98%, s__Shigella sonnei..."
1,s__Klebsiella pneumoniae,14975,7,"s__Klebsiella pneumoniae 99.73%, s__(g__) 0.21..."
2,s__Staphylococcus aureus,14959,3,"s__Staphylococcus aureus 99.64%, s__(g__) 0.35..."
3,s__Salmonella enterica,13832,2,"s__Salmonella enterica 97.27%, s__(g__) 2.73%"
4,s__Streptococcus pneumoniae,8895,2,"s__Streptococcus pneumoniae 99.99%, s__Haemoph..."
...,...,...,...,...
71940,s__Alistipes sp015059865,1,1,s__(g__) 100.0%
71941,s__Caproiciproducens sp900546895,1,1,s__(g__) 100.0%
71942,s__RGIG8775 sp017621015,1,1,s__(g__) 100.0%
71943,s__S144-17 sp015486425,1,1,s__(g__) 100.0%


In [19]:
# for release 214!!! Change files for the new releases
rel = '214'
convB = pd.read_excel('input/gtdb_vs_ncbi_r'+str(rel)+'_bacteria.xlsx', sheet_name='Species')
print('Unique bacteria species by gtdb - ',len(convB['GTDB R214 species'].unique()))

convB.rename(columns = {'GTDB R214 species':'sp_gtdb',"List of NCBI species":'ncbi'}, inplace = True)
new = convB.ncbi.str.rsplit(' ', 1, expand=True)\
  .rename(columns=lambda x: 'col{}'.format(x + 1))
convB['sp_ncbi'] = new['col1']
convB['sp_ncbi'] = convB['sp_ncbi'].str.replace(r's__', '')
convB['sp_gtdb'] = convB['sp_gtdb'].str.replace(r's__', '')
convB = convB[['sp_gtdb','sp_ncbi']]
convB = convB.loc[convB['sp_ncbi'] != "(g__)"]
convB = convB.reset_index(drop=True)

convB.tail(2)

Unique bacteria species by gtdb -  71945


/var/folders/5j/pvj6k02n3mn9ztpn77vjtfzw0000gn/T/ipykernel_85898/2449744670.py:7: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  new = convB.ncbi.str.rsplit(' ', 1, expand=True)\


,sp_gtdb,sp_ncbi
11970,Bacillus_A pseudomycoides_B,Bacillus pseudomycoides
11971,Dyella_B choica,Dyella choica


In [16]:
convA = pd.read_excel('input/gtdb_vs_ncbi_r214_archaea.xlsx', sheet_name='Species')
print('Unique archaea species by gtdb - ',len(convA['GTDB R214 species'].unique()))

convA['top_sp_ncbi'] = convA['List of NCBI species'].str.split(',', expand=True)[0]
convA['GTDB R214 species'] = convA['GTDB R214 species'].str.replace(r's__', '')
convA['top_sp_ncbi'] = convA['top_sp_ncbi'].str.replace(r's__', '')
convA.rename(columns = {'GTDB R214 species':'sp_gtdb'}, inplace = True)
new = convA.top_sp_ncbi.str.rsplit(' ', 1, expand=True)\
  .rename(columns=lambda x: 'col{}'.format(x + 1))
convA['sp_ncbi'] = new['col1']

convA = convA[['sp_gtdb','sp_ncbi']]
convA = convA.loc[convA['sp_ncbi'] != "(g__)"]
convA = convA.reset_index(drop=True)

convA.tail(2)

Unique archaea species by gtdb -  4043


/var/folders/5j/pvj6k02n3mn9ztpn77vjtfzw0000gn/T/ipykernel_85898/1251250436.py:8: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  new = convA.top_sp_ncbi.str.rsplit(' ', 1, expand=True)\


,sp_gtdb,sp_ncbi
240,Methanospirillum hungatei_A,Methanospirillum hungatei
241,Methanohalarchaeum thermophilum,Candidatus Methanohalarchaeum thermophilum


In [20]:
frames = [convA,convB]
conv = pd.concat(frames)
conv = conv.reset_index(drop=True)

print('Unique species by gtdb - ',len(conv['sp_gtdb'].unique()))
conv.tail(2)

Unique species by gtdb -  12214


,sp_gtdb,sp_ncbi
12212,Bacillus_A pseudomycoides_B,Bacillus pseudomycoides
12213,Dyella_B choica,Dyella choica


In [21]:
gtdbN = pd.merge(gtdb, conv, how="inner", left_on="sp", right_on='sp_gtdb').sort_values('sp_ncbi').reset_index(drop=True)
gtdbN.to_excel('Tax_gtdb-ncbi_v2.xlsx', index=False)

gtdbN.head(2)

,domain,phylum,class,order,family,genus,sp,sp_gtdb,sp_ncbi
0,Bacteria,Pseudomonadota,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas_E,Pseudomonas_E mandelii_C,Pseudomonas_E mandelii_C,"(g__) 25.0%, Pseudomonas mandelii 25.0%, Pseud..."
1,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Neisseriaceae,Neisseria,Neisseria sp000186165,Neisseria sp000186165,"(g__) 30.3%, Neisseria subflava 18.18%, Neisse..."


### Random check merge table

In [68]:
gtdbN.loc[gtdbN['sp_ncbi'] == 'Treponema succinifaciens']

,domain,phylum,class,order,family,genus,sp,sp_gtdb,sp_ncbi
11343,Bacteria,Spirochaetota,Spirochaetia,Treponematales,Treponemataceae,Treponema_D,Treponema_D succinifaciens,Treponema_D succinifaciens,Treponema succinifaciens
